In [1]:
import pandas as pd
import os

# Function to extract reasons from Excel files
def extract_reasons_from_excel(folder_path):
    reasons = set()  # Using a set to avoid duplicates
    # Loop through all Excel files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            # Read the Excel file
            try:
                df = pd.read_excel(file_path, header=None)
                # Loop through each cell in the DataFrame to find reasons
                for row in df.itertuples(index=False):
                    for cell in row:
                        if isinstance(cell, str) and len(cell) > 0:
                            # Assuming reasons are located towards the end of the rows
                            if any(keyword in cell for keyword in ['sale', 'condition', 'property', 'land']):
                                reasons.add(cell.strip())
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    
    return list(reasons)

# Specify the folder containing the Excel files
folder_path = "Tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions"
extracted_reasons = extract_reasons_from_excel(folder_path)

# Print the unique reasons
print("Unique Reasons Found:")
for reason in extracted_reasons:
    print(reason)

# Save the unique reasons to a CSV file
output_csv_path = 'extracted_reasons.csv'
pd.DataFrame(extracted_reasons, columns=['Reasons']).to_csv(output_csv_path, index=False)

print(f"\nUnique Reasons saved to {output_csv_path}")


Unique Reasons Found:
Auction sale "as is"
Combination after sale
L-Multiple parcels on one sale; L-Multiple parcels on one sale
Investment property - rental
2 sales on one day
OUTSIDE OF TIMEFRAME / Not used due to lack of sales for group (only 2 sales) (now com class)
Extreme outlier; Twp. Land sales range $22K-$45K
H-Sold twice in one year; L-Multiple parcels on one sale
Dwelling in poor condition - purchased by investor - flip sale
Property substantially updated prior to sale (physical change)
Split off of parcel after the sale (split/cut)
3 sales on one day
Unadvertised sale to Investor/Realtor
significant change. New construction. Combine after sale
Signficant physical changes after sale
Parcel combined with other after sale
Not typical sale for the neighborhood
physical change before sale
land order done this year
Estate sale, not an arm's length transaction
Remodeled and flipped after $53,000 sale 8/30/22
Out of state sale
unlisted / dwlg renovation after sale
Sold for under 10

In [2]:
import pandas as pd
from Levenshtein import distance as levenshtein_distance

def cluster_reasons_by_levenshtein(csv_file_path, threshold=3):
    # Read reasons from the CSV file
    df = pd.read_csv(csv_file_path)
    
    if df.empty:
        print("The CSV file is empty.")
        return []
    
    # Assuming reasons are in the first column
    reasons = df.iloc[:, 0].tolist()  # Adjust the index if necessary

    # Group similar reasons based on Levenshtein distance
    groups = []
    visited = set()  # To keep track of visited reasons
    
    for reason in reasons:
        if reason in visited:
            continue
        
        # Create a new group with the current reason
        current_group = [reason]
        visited.add(reason)

        for other_reason in reasons:
            if other_reason not in visited and levenshtein_distance(reason, other_reason) <= threshold:
                current_group.append(other_reason)
                visited.add(other_reason)
        
        groups.append(current_group)
    
    return groups

# Example usage
csv_file_path = 'extracted_reasons.csv'  # Specify the path to your CSV file
threshold = 3  # Adjust as needed
clusters = cluster_reasons_by_levenshtein(csv_file_path, threshold)

# Print the clusters
print("Clusters of Similar Reasons:")
for i, cluster in enumerate(clusters):
    print(f"Cluster {i + 1}: {cluster}")


Clusters of Similar Reasons:
Cluster 1: ['Auction sale "as is"']
Cluster 2: ['Combination after sale']
Cluster 3: ['L-Multiple parcels on one sale; L-Multiple parcels on one sale']
Cluster 4: ['Investment property - rental']
Cluster 5: ['2 sales on one day', '3 sales on one day']
Cluster 6: ['OUTSIDE OF TIMEFRAME / Not used due to lack of sales for group (only 2 sales) (now com class)']
Cluster 7: ['Extreme outlier; Twp. Land sales range $22K-$45K']
Cluster 8: ['H-Sold twice in one year; L-Multiple parcels on one sale']
Cluster 9: ['Dwelling in poor condition - purchased by investor - flip sale']
Cluster 10: ['Property substantially updated prior to sale (physical change)']
Cluster 11: ['Split off of parcel after the sale (split/cut)']
Cluster 12: ['Unadvertised sale to Investor/Realtor']
Cluster 13: ['significant change. New construction. Combine after sale']
Cluster 14: ['Signficant physical changes after sale', 'significant physical changes after sale', 'Significant Physical Changes

In [6]:
import re
import pandas as pd
from Levenshtein import distance as levenshtein_distance

def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.strip()  # Remove leading/trailing whitespace
    return text

def cluster_reasons_by_levenshtein(csv_file_path, threshold=5):
    # Read reasons from the CSV file
    df = pd.read_csv(csv_file_path)
    
    if df.empty:
        print("The CSV file is empty.")
        return []
    
    # Assuming reasons are in the first column
    reasons = df.iloc[:, 0].dropna().apply(normalize_text).tolist()

    # Group similar reasons based on Levenshtein distance
    groups = []
    visited = set()  # To keep track of visited reasons
    
    for reason in reasons:
        if reason in visited:
            continue
        
        # Create a new group with the current reason
        current_group = [reason]
        visited.add(reason)

        for other_reason in reasons:
            if other_reason not in visited and levenshtein_distance(reason, other_reason) <= threshold:
                current_group.append(other_reason)
                visited.add(other_reason)
        
        groups.append(current_group)
    
    return groups

# Example usage
csv_file_path = 'extracted_reasons.csv'  # Specify the path to your CSV file
threshold = 50  # Try increasing this value to group more similar reasons together
clusters = cluster_reasons_by_levenshtein(csv_file_path, threshold)

# Print the clusters
print("Clusters of Similar Reasons:")
for i, cluster in enumerate(clusters):
    print(f"Cluster {i + 1}: {cluster}")


Clusters of Similar Reasons:
Cluster 1: ['auction sale as is', 'combination after sale', 'lmultiple parcels on one sale lmultiple parcels on one sale', 'investment property  rental', '2 sales on one day', 'extreme outlier twp land sales range 22k45k', 'hsold twice in one year lmultiple parcels on one sale', 'dwelling in poor condition  purchased by investor  flip sale', 'property substantially updated prior to sale physical change', 'split off of parcel after the sale splitcut', '3 sales on one day', 'unadvertised sale to investorrealtor', 'significant change new construction combine after sale', 'signficant physical changes after sale', 'parcel combined with other after sale', 'not typical sale for the neighborhood', 'physical change before sale', 'land order done this year', 'estate sale not an arms length transaction', 'remodeled and flipped after 53000 sale 83022', 'out of state sale', 'unlisted  dwlg renovation after sale', 'sold for under 10000investment property', 'significant c